In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def compare_resource_usage(query, run, fname):
    memory_q1_opt = pd.read_csv(f'{query}/opt/resource-usage-{run}.csv')
    memory_q1_ref = pd.read_csv(f'{query}/ref/resource-usage-{run}.csv')
    
    plt.plot(memory_q1_ref['memory_used'] / (1000 * 1000 * 1000))
    plt.plot(memory_q1_opt['memory_used'] / (1000 * 1000 * 1000))
    plt.savefig(f'memory-{fname}.eps')
    plt.show()

    plt.plot(memory_q1_ref['cpu_used'])
    plt.plot(memory_q1_opt['cpu_used'])
    plt.show()

    N = 15
    plt.plot(np.convolve(memory_q1_ref['cpu_used'], np.ones(N)/N, mode='valid'))
    plt.plot(np.convolve(memory_q1_opt['cpu_used'], np.ones(N)/N, mode='valid'))
    plt.savefig(f'cpu15s-{fname}.eps')
    plt.show()

In [ ]:
compare_resource_usage('benchmark-results-lsqb/q1.sql', '05', 'q1-sf100')

#compare_resource_usage('benchmark-results-lsqb-sf100/q4.sql')

In [102]:
def gen_stats(resdir, ignore_first = True, results='opt'):
    df_opt = pd.read_csv(f'{resdir}/results-{results}.csv')
    if ignore_first:
        df_opt = df_opt[df_opt['run'] != 1]
    df_opt = df_opt.groupby(['query', 'physical_cj']).agg({'runtime': ['mean', 'std', 'sum']})
    df_opt.to_csv(f'{resdir}/results-{results}-mean.csv')
    return df_opt
def gen_stats_ref(resdir):
    stats_opt_df = pd.read_csv(f'{resdir}/results-ref.csv')
    df_opt = stats_opt_df[stats_opt_df['run'] != 1]
    df_opt = df_opt.groupby(['query']).agg({'runtime': ['mean', 'std', 'sum']})
    df_opt.to_csv(f'{resdir}/results-ref-mean.csv')
    return df_opt
def gen_stats_imdb(resdir):
    stats_opt_df = pd.read_csv(f'{resdir}/results-opt.csv')
    df_opt = stats_opt_df[stats_opt_df['run'] != 1]
    df_opt['query'] = df_opt['query'].replace(r'[a-z].sql', '.sql', regex=True)
    df_opt = df_opt.groupby(['query']).agg({'runtime': ['mean', 'std', 'sum']})
    df_opt.to_csv(f'{resdir}/results-opt-mean.csv')
    return df_opt

In [26]:
df_opt = gen_stats('benchmark-results-stats')
df_opt.sum()

runtime  mean     77.824798
         std      19.531498
         sum     389.123991
dtype: float64

In [27]:
df_opt = gen_stats('benchmark-results-stats-hint')
df_opt.sum()

runtime  mean     57.936630
         std       5.402731
         sum     289.683150
dtype: float64

In [47]:
df_opt = gen_stats_imdb('benchmark-results-imdb')
#df_opt['query'] = df_opt['query'].replace(r'\d\\.sql', '.sql', regex=True)
df_opt

/tmp/ipykernel_38519/3726565282.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_opt['query'] = df_opt['query'].replace(r'[a-z].sql', '.sql', regex=True)


runtime                       
             mean       std          sum
query                                   
17.sql  35.692412  0.586494  1070.772355
2.sql    4.720473  0.222184    94.409469
20.sql  21.720675  1.919547   325.810121
3.sql    4.696670  0.239623    70.450057
5.sql    1.015987  0.975145    15.239801

In [69]:
df_opt = gen_stats('benchmark-results-lsqb')
df_opt

runtime                        
                               mean        std          sum
query       physical_cj                                    
q4-hint.sql True         620.640559  18.049639  1241.281119
q4.sql      True         585.531344   2.172676  1171.062687

In [123]:
df_opt = gen_stats('benchmark-results-lsqb', results='ref', ignore_first=False)
df_opt.round(decimals=2)

runtime                 
                       mean     std      sum
query  physical_cj                          
q1.sql False        3095.81  232.44  6191.62

In [78]:
df_ref = gen_stats_ref('benchmark-results-lsqb')
df_ref

runtime                        
            mean        std          sum
query                                   
q4.sql  602.2586  37.050436  3011.293002

In [86]:
df_opt = gen_stats('benchmark-results-tpch-sf100')
df_opt.round(decimals=2)

runtime              
                                 mean   std     sum
query             physical_cj                      
11-hint.sql       False        164.58  5.90  822.88
                  True         166.96  4.60  834.82
11.sql            False        162.79  3.84  813.94
                  True         167.86  5.79  839.30
2.sql             True          81.54  2.66  407.68
median-1-hint.sql False         50.74  1.90  253.71
                  True          49.67  1.15  248.37
median-1.sql      False         50.04  1.34  250.18
                  True          51.56  0.89  257.78

In [116]:
df_opt = gen_stats('benchmark-results-tpch')
df_opt.round(decimals=2)

runtime                
                                 mean    std      sum
query             physical_cj                        
11-hint.sql       False        346.31   9.74  1731.57
                  True         344.01  10.95  1720.04
11.sql            False        334.92   8.20  1674.62
                  True         329.42   5.92  1647.09
2.sql             False        164.21   4.74   821.07
                  True         160.64   3.74   803.22
median-1-hint.sql False        106.18   1.12   530.92
                  True         102.04   3.88   510.18
median-1.sql      False        107.51   8.90   537.53
                  True         105.11   3.86   525.53

In [111]:
df_opt = gen_stats('benchmark-results-tpch', results='ref')
df_opt.round(decimals=2)

runtime                
                                 mean    std      sum
query             physical_cj                        
11-hint.sql       False        351.76  14.56  1758.79
11.sql            False        351.14  11.91  1755.69
2.sql             False        179.43   6.50   897.17
median-1-hint.sql False        165.93   2.50   829.66
median-1.sql      False        168.44   4.41   842.21

In [51]:
df_opt = gen_stats('benchmark-results-tpch-part1')
df_opt

runtime                      
                         mean       std         sum
query                                              
11-hint.sql        189.647959  3.507685  948.239795
11.sql                    NaN       NaN    0.000000
2.sql               79.777111  3.000262  398.885554
median-1-hint.sql   52.460698  1.423260  262.303489
median-1.sql        91.899140  2.280504  459.495702

In [55]:
df_opt = gen_stats('benchmark-results-snap')
df_opt.round(decimals=2)

runtime              
                      mean   std     sum
query                                   
dblp-path02.sql       1.31  0.18    6.55
dblp-path03.sql       1.59  0.12    7.97
dblp-path04.sql       1.76  0.16    8.82
dblp-path05.sql       2.03  0.25   10.16
dblp-path06.sql       2.18  0.14   10.92
dblp-path07.sql       2.38  0.26   11.92
dblp-path08.sql       2.53  0.30   12.64
dblp-tree01.sql       1.47  0.28    7.37
dblp-tree02.sql       1.69  0.16    8.46
dblp-tree03.sql       1.99  0.16    9.93
google-path02.sql     5.10  0.52   25.48
google-path03.sql     6.08  0.65   30.39
google-path04.sql     6.89  0.30   34.46
google-path05.sql     7.53  0.48   37.67
google-path06.sql     8.80  0.25   43.98
google-path07.sql     9.76  1.21   48.80
google-path08.sql    10.05  1.49   50.26
google-tree01.sql     6.53  1.11   32.65
google-tree02.sql     7.29  0.73   36.44
google-tree03.sql     8.16  0.66   40.82
patents-path02.sql   12.03  0.70   60.14
patents-path03.sql   13.01  0.58   65.06
patents-path04.sql   14.36  0.59   71.82
patents-path05.sql   15.32  1.09   76.58
patents-path06.sql   15.97  1.15   79.85
patents-path07.sql   17.66  0.56   88.32
patents-path08.sql   18.47  1.20   92.35
patents-tree01.sql   14.01  0.75   70.06
patents-tree02.sql   14.38  1.19   71.89
patents-tree03.sql   16.07  1.32   80.35
wiki-path02.sql      21.38  0.68  106.91
wiki-path03.sql      23.71  0.54  118.56
wiki-path04.sql      25.94  1.12  129.70
wiki-path05.sql      27.46  0.64  137.32
wiki-path06.sql      30.16  1.00  150.80
wiki-path07.sql      33.32  1.62  166.61
wiki-path08.sql      34.49  0.68  172.43
wiki-tree01.sql      25.44  0.43  127.20
wiki-tree02.sql      27.64  0.57  138.19
wiki-tree03.sql      30.70  1.01  153.49

In [148]:
pd.set_option('display.max_rows', 500)
df = gen_stats('benchmark-results-hetio', ignore_first=False)
df.round(decimals=2)

runtime             
                                   mean   std    sum
query               physical_cj                     
2-01-CbGaD.sql      False          0.51   NaN   0.51
                    True           0.39  0.02   1.17
2-02-CdGuD.sql      False          0.80   NaN   0.80
                    True           0.66  0.08   1.98
2-03-CrCtD.sql      False          0.32   NaN   0.32
                    True           0.29  0.03   0.88
2-04-CtDrD.sql      False          0.33   NaN   0.33
                    True           0.23  0.02   0.70
2-05-CuGdD.sql      False          0.85   NaN   0.85
                    True           0.62  0.06   1.85
3-01-CbGbCtD.sql    False          0.54   NaN   0.54
                    True           0.47  0.05   1.41
3-02-CbGeAlD.sql    False          2.34   NaN   2.34
                    True           1.51  0.10   3.02
3-03-CbGiGaD.sql    False          1.50   NaN   1.50
                    True           0.93   NaN   0.93
3-04-CcSEcCtD.sql   False          1.16   NaN   1.16
                    True           0.90   NaN   0.90
3-05-CdGdCtD.sql    False          1.06   NaN   1.06
                    True           0.70   NaN   0.70
3-06-CdGuCtD.sql    False          0.79   NaN   0.79
                    True           0.65   NaN   0.65
3-07-CiPCiCtD.sql   False          2.92   NaN   2.92
                    True           4.84   NaN   4.84
3-08-CpDpCtD.sql    False          0.33   NaN   0.33
                    True           0.35   NaN   0.35
3-09-CrCrCtD.sql    False          0.39   NaN   0.39
                    True           0.38   NaN   0.38
3-10-CrCbGaD.sql    False          0.41   NaN   0.41
                    True           0.51   NaN   0.51
3-11-CtDdGdD.sql    False          1.02   NaN   1.02
                    True           0.60   NaN   0.60
3-12-CtDdGuD.sql    False          0.78   NaN   0.78
                    True           0.56   NaN   0.56
3-13-CtDlAlD.sql    False          0.33   NaN   0.33
                    True           0.31   NaN   0.31
3-14-CtDpSpD.sql    False          0.50   NaN   0.50
                    True           0.30   NaN   0.30
3-15-CtDuGdD.sql    False          0.94   NaN   0.94
                    True           0.74   NaN   0.74
3-16-CtDuGuD.sql    False          0.89   NaN   0.89
                    True           0.68   NaN   0.68
3-17-CuGdCtD.sql    False          0.80   NaN   0.80
                    True           0.71   NaN   0.71
3-18-CuGuCtD.sql    False          0.76   NaN   0.76
                    True           0.64   NaN   0.64
4-01-CbGiGiGaD.sql  False          1.28   NaN   1.28
                    True           1.35   NaN   1.35
4-02-CbGpBPpGaD.sql False          3.95   NaN   3.95
                    True           4.29   NaN   4.29
4-03-CbGpPWpGaD.sql False          3.86   NaN   3.86
                    True           2.44   NaN   2.44
CbGiGiGaD.sql       False          1.40  0.17   7.00
                    True           1.35  0.22   6.77
CbGpBPpGaD.sql      False          3.83  0.31  30.64
                    True           3.43  0.54  27.46
CbGpPWpGaD.sql      False          3.02  0.35  24.13
                    True           2.71  0.30  21.65
CtDpSpD.sql         False          0.42  0.07   1.27
                    True           0.35  0.05   1.06
CtDuGdD.sql         False          0.85  0.16   2.56
                    True           0.62  0.03   1.86
CtDuGuD.sql         False          0.86  0.09   2.58
                    True           0.61  0.06   1.83
CuGdCtD.sql         False          0.77  0.22   2.32
                    True           0.64  0.07   1.91
CuGuCtD.sql         False          1.01  0.06   3.04
                    True           0.66  0.10   1.98
q1.sql              False          3.36  0.49  20.16
                    True           2.78  0.43  16.70

In [149]:
df = gen_stats('benchmark-results-hetio', results='ref', ignore_first=False)
df.round(decimals=2)

runtime               
                                   mean    std     sum
query               physical_cj                       
2-01-CbGaD.sql      False          0.36    NaN    0.36
2-02-CdGuD.sql      False          0.66    NaN    0.66
2-03-CrCtD.sql      False          0.21    NaN    0.21
2-04-CtDrD.sql      False          0.18    NaN    0.18
2-05-CuGdD.sql      False          0.57    NaN    0.57
3-01-CbGbCtD.sql    False          0.57    NaN    0.57
3-02-CbGeAlD.sql    False          2.06    NaN    2.06
3-03-CbGiGaD.sql    False          0.53    NaN    0.53
3-04-CcSEcCtD.sql   False         12.21    NaN   12.21
3-05-CdGdCtD.sql    False          1.14    NaN    1.14
3-06-CdGuCtD.sql    False          0.56    NaN    0.56
3-07-CiPCiCtD.sql   False          0.85    NaN    0.85
3-08-CpDpCtD.sql    False          0.25    NaN    0.25
3-09-CrCrCtD.sql    False          0.31    NaN    0.31
3-10-CrCbGaD.sql    False          0.32    NaN    0.32
3-11-CtDdGdD.sql    False          0.79    NaN    0.79
3-12-CtDdGuD.sql    False          0.60    NaN    0.60
3-13-CtDlAlD.sql    False          0.22    NaN    0.22
3-14-CtDpSpD.sql    False          0.23    NaN    0.23
3-15-CtDuGdD.sql    False          0.59    NaN    0.59
3-16-CtDuGuD.sql    False          0.54    NaN    0.54
3-17-CuGdCtD.sql    False          0.49    NaN    0.49
3-18-CuGuCtD.sql    False          0.57    NaN    0.57
4-01-CbGiGiGaD.sql  False         19.01    NaN   19.01
4-02-CbGpBPpGaD.sql False         52.92    NaN   52.92
4-03-CbGpPWpGaD.sql False         11.18    NaN   11.18
CbGiGiGaD.sql       False         16.43   1.64   82.17
CbGpBPpGaD.sql      False         58.57  12.98  468.56
CbGpPWpGaD.sql      False         12.75   1.29  102.01
CtDpSpD.sql         False          0.23   0.01    0.70
CtDuGdD.sql         False          0.65   0.07    1.95
CtDuGuD.sql         False          0.68   0.10    2.04
CuGdCtD.sql         False          0.52   0.03    1.55
CuGuCtD.sql         False          0.85   0.15    2.56
q1.sql              False         12.10   1.64   72.58

In [ ]:
stats_ref_df = pd.read_csv('benchmark-results-stats/results-ref.csv')
df_ref = stats_ref_df[stats_opt_df['run'] != 1]
df_ref = df_ref.groupby(['query']).agg({'runtime': ['mean', 'std', 'sum']})
df_ref.to_csv('benchmark-results-stats/results-ref-mean.csv')

In [ ]:
#df[['query', 'runtime']].plot()
df = df_opt
plt.plot(df[df.index.str.contains('hint.sql')][[('runtime', 'mean')]])
plt.plot(df[~df.index.str.contains('hint.sql')][[('runtime', 'mean')]])
#df.index#['query']

In [ ]:
df_opt[df_opt.index.str.contains('hint.sql')][[('runtime', 'sum')]].sum()

In [ ]:
df_opt[~df_opt.index.str.contains('hint.sql')][[('runtime', 'sum')]].sum()

In [ ]:
df_ref[[('runtime', 'sum')]].sum()

In [ ]:
df = pd.read_csv('benchmark-results-tpch/results-ref.csv')

In [ ]:
print(df)